##### import dataset et modèle

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
import plotly.express as px

# 1. Chargement des données

X_full = pd.read_parquet("dataset_full.parquet") 

# 2. Chargement du modèle
loaded_model = xgb.Booster()
loaded_model.load_model("xgboost_v2.json")

print(f"✅ Modèle et {len(X_full)} entreprises chargés.")

---

##### Test de prédictions

In [ ]:
# --- ÉTAPE A : PRÉPARATION & PRÉDICTION ---

# 1. Copie pour ne pas modifier l'original
X_input = X_full.copy()

# 2. Feature Engineering (Recréation des variables du modèle)
if "Economie sociale et solidaire unité légale" in X_input.columns:
    X_input['is_ess'] = X_input["Economie sociale et solidaire unité légale"].map({'O': 1, 'N': 0}).fillna(0)

# 3. One-Hot Encoding des secteurs
if 'libelle_section_ape' in X_input.columns:
    X_input = pd.get_dummies(X_input, columns=['libelle_section_ape'], prefix='APE')

# 4. Alignement strict avec les attentes du modèle (Colonnes manquantes et ordre)
for col in loaded_model.feature_names:
    if col not in X_input.columns:
        X_input[col] = 0

X_input = X_input[loaded_model.feature_names]

# 5. Calcul des prédictions brutes
dmatrix_full = xgb.DMatrix(X_input)
raw_preds = loaded_model.predict(dmatrix_full)

# --- ÉTAPE B : SCORING ET ANALYSE MÉTIER ---

# 1. Calcul de l'Indice de Risque Global (0-100)
risk_scores = pd.Series(raw_preds).rank(pct=True, ascending=False) * 100

# 2. Création du DataFrame de résultats consolidé
df_resultats = pd.DataFrame({
    'SIREN': X_full["SIREN"],
    'Dénomination': X_full["Dénomination de l'unité légale"],
    'Indice_Risque': risk_scores.values
}, index=X_full.index)

# 3. Fonction de calcul des probabilités par horizon avec nettoyage
def calculer_probabilites_propres(score):

    p_1an = 1 / (1 + np.exp(-(score - 85) / 3))
    p_2ans = 1 / (1 + np.exp(-(score - 70) / 5))
    p_3ans = 1 / (1 + np.exp(-(score - 55) / 7))
    
    probas = []
    for p in [p_1an, p_2ans, p_3ans]:
        val = round(p * 100, 2)
        probas.append(val if val >= 0.01 else 0.0)
    return probas[0], probas[1], probas[2]

# 4. Application des calculs d'horizons
df_resultats['Prob_1an'], df_resultats['Prob_2ans'], df_resultats['Prob_3ans'] = zip(*df_resultats['Indice_Risque'].map(calculer_probabilites_propres))

# 5. Attribution du Statut Expert
df_resultats['Statut_Expert'] = pd.cut(
    df_resultats['Indice_Risque'], 
    bins=[0, 55, 70, 85, 100], 
    labels=['🟢 SAIN', '🟡 OBSERVATION', '🟠 VIGILANCE', '🔴 CRITIQUE'],
    include_lowest=True
)

# 6. Affichage des résultats prioritaires
print(f"✅ Analyse terminée sur {len(df_resultats)} entreprises.")
cols_view = ['SIREN', 'Dénomination', 'Indice_Risque', 'Prob_1an', 'Prob_2ans', 'Prob_3ans', 'Statut_Expert']
display(df_resultats[cols_view].sort_values('Indice_Risque', ascending=False).head(15))

In [ ]:
# 6. Affichage final
print("✅ Diagnostic de survie consolidé terminé (Chiffres nettoyés) !")
cols_view = ['SIREN', 'Dénomination', 'Indice_Risque', 'Prob_1an', 'Prob_2ans', 'Prob_3ans', 'Statut_Expert']
display(df_resultats[cols_view].sort_values('Indice_Risque', ascending=False).tail(10))

In [ ]:
df_resultats.shape

In [ ]:
X_full.head()

---

#### Fusion des résultats avec le dataset de départ

In [ ]:
# On retire 'SIREN' et 'Dénomination' de X_full lors de la fusion (déjà présents dans df_resultat)

cols_brutes = X_full.drop(columns=["SIREN", "Dénomination de l'unité légale"])

# Fusion (Join utilise l'index par défaut)
df_final_complet = df_resultats.join(cols_brutes)

# Réorganisation pour avoir le SIREN et le Score en premier, puis le reste
cols = ['SIREN', 'Dénomination', 'Statut_Expert', 'Indice_Risque', 'Prob_1an', 'Prob_2ans', 'Prob_3ans']
autres_cols = [c for c in df_final_complet.columns if c not in cols]
df_final_complet = df_final_complet[cols + autres_cols]

print(f"✅ Fusion réussie : {df_final_complet.shape[1]} colonnes disponibles.")

In [ ]:
df_final_complet.head()

In [ ]:
# 1. Préparation des données pour le graph
df_plot = resume_risques.reset_index()
df_plot.columns = ['Statut', 'Nombre de sociétés', 'Pourcentage']

# 2. Création du graphique interactif
fig = px.bar(
    df_plot, 
    x='Statut', 
    y='Nombre de sociétés',
    text='Pourcentage',
    color='Statut',
    color_discrete_map={
        '🟢 SAIN': '#2ecc71',
        '🟡 OBSERVATION': '#f1c40f',
        '🟠 VIGILANCE': '#e67e22',
        '🔴 CRITIQUE': '#e74c3c'
    },
    title="Analyse du Risque Portefeuille",
    labels={'Statut': 'Catégorie de Risque', 'Nombre de sociétés': 'Nombre d\'entreprises'}
)

# 3. Personnalisation du style
fig.update_traces(
    texttemplate='%{text:.2f}%', 
    textposition='outside',
    hovertemplate="<b>%{x}</b><br>Effectif: %{y}<br>Part: %{text:.2f}%<extra></extra>"
)

fig.update_layout(
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0)',
    yaxis_title="Nombre d'entreprises",
    font=dict(size=14),
    height=600
)

fig.show()

In [ ]:
df_final_complet.dtypes

In [ ]:
# # Export au format Parquet
# df_final_complet.to_parquet("Reporting_Risques_Survie_2026.parquet", compression='snappy')

# print("💾 Fichier Parquet sauvegardé !")

In [ ]:
# Taux de risque par département
stats_dep = df_final_complet.groupby('Code du département de l\'établissement')['Statut_Expert'].value_counts(normalize=True).unstack()
stats_dep['Taux_Alerte'] = (stats_dep['🔴 CRITIQUE'] + stats_dep['🟠 VIGILANCE']) * 100
display(stats_dep.sort_values('Taux_Alerte', ascending=False).head(5))

In [ ]:
# 1. On prépare les données
df_map = stats_dep.reset_index()

# 2. Création de la carte avec une échelle plus nuancée (RdYlGn inversé : Vert -> Jaune -> Rouge)
fig_map = px.choropleth(
    df_map,
    geojson='https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements-version-simplifiee.geojson',
    locations='Code du département de l\'établissement',
    featureidkey="properties.code",
    color='Taux_Alerte',
    color_continuous_scale="RdYlGn_r", 
    range_color=(df_map['Taux_Alerte'].min(), df_map['Taux_Alerte'].max()),
    scope='europe',
    title="Nuancier du Risque de Défaillance par Département",
    labels={'Taux_Alerte': 'Taux (%)'}
)

# 3. Optimisation du rendu
fig_map.update_geos(fitbounds="locations", visible=False)
fig_map.update_layout(
    margin={"r":0,"t":50,"l":0,"b":0},
    coloraxis_colorbar=dict(
        title="Niveau de Risque",
        ticksuffix="%",
        thicknessmode="pixels", thickness=15,
        lenmode="pixels", len=200,
    )
)

fig_map.show()